In [1]:
import importlib as il
import numpy as np
import h5py

from ase.calculators.emt import EMT
from ase.io import read
from hilde.helpers.supercell import make_cubic_supercell
from hilde.tasks.calculate import calculate_multiple
from hilde.structure.io import inform

In [2]:
ph3 = il.import_module('hilde.phono3py.phono3')

In [3]:
atoms = read('../Al.in', format='aims')

In [4]:
sc2, smatrix2 = make_cubic_supercell(atoms, 64)
sc3, smatrix3 = make_cubic_supercell(atoms, 8)
inform(sc3)

**Warning: Cubicness of supercell is 0.690 (0.329)
**-> Sytems: Al, target size 8

Geometry info for:
  input geometry:    Al9
  Number of atoms:   9
  Species:           Al
  Periodicity:       [ True  True  True]
  Lattice:  
    [ 6.070899  0.       -2.023633]
    [ 0.        6.070899 -2.023633]
    [0.       0.       4.047266]

  supercell_matrix: [-2, 1, 2, 1, -2, 2, 1, 1, -1]

Cell lengths and angles [Å, °]:
  a, b, c:      6.3993      6.3993      4.0473
  α, β, γ:    108.4349    108.4349     84.2608
  Volume:     149.1653 Å**3


In [5]:
phono3py_settings = {
    'atoms': atoms,
    'fc2_supercell_matrix': smatrix2,
    'fc3_supercell_matrix': smatrix3,
    'cutoff_pair_distance': 3
}

In [6]:
il.reload(ph3)
# help(ph3.preprocess)

<module 'hilde.phono3py.phono3' from '/home/knoop/local/hilde/hilde/phono3py/phono3.py'>

In [7]:
phonon3, sc2, sc3, scs2, scs3 = ph3.preprocess(**phono3py_settings)

In [8]:
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 2
Number of FC3 displacements: 31
Number of FC3 displacements that were created: 31


In [9]:
calc = EMT()

## Compute Forces

In [10]:
scs2_computed = calculate_multiple(scs2, calc, 'test_run/fc2', trajectory='fc2.traj')
fc2_forces = ph3.get_forces(scs2_computed)

/home/knoop/local/hilde/hilde/tasks/calculate.py:46: UserWarning: dont compare stupid things -> replace me with hashes
  warn("dont compare stupid things -> replace me with hashes")


In [11]:
%time scs3_computed = calculate_multiple(scs3, calc, 'test_run/fc3', trajectory='fc3.traj')
fc3_forces = ph3.get_forces(scs3_computed)
len(scs3_computed), len(scs3)

CPU times: user 749 ms, sys: 16 ms, total: 765 ms
Wall time: 764 ms


(31, 31)

In [12]:
[s for s in scs3_computed if s][-1].info

{'displacement_id': 32}

## Produce Force Constants

In [13]:
phonon3.produce_fc2(fc2_forces)

In [14]:
%time phonon3.produce_fc3(fc3_forces)

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0071  0.0071  0.0283]
Expanding fc3
Cutting-off fc3 (cut-off distance: 3.000000)
Building atom mapping table...
Creating contracted fc3...
CPU times: user 244 ms, sys: 534 ms, total: 778 ms
Wall time: 136 ms


In [15]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [16]:
q_mesh = 3*[3]
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=q_mesh,
                                   log_level=0)

In [17]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

In [18]:
with h5py.File('kappa-m{}.hdf5'.format(''.join(map(str, q_mesh)))) as kappa:
    print(kappa['frequency'][:])
    print(kappa['kappa'][:][np.where(kappa['temperature'][:] == 300)][:3])
    # assert 5 < kappa['frequency'][:].max() < 10, kappa['frequency'][:].max()
    # assert 25 < kappa['kappa'][:][30][:3].mean() < 35

[[-7.92231114e-08  1.25433682e-06  1.27761432e-06]
 [-6.18972256e+00  1.46177915e-01  2.63121323e+00]
 [-6.16949440e+00  1.89411179e+00  5.26852675e+00]
 [-6.70417613e+00  1.00875939e+00  5.88649436e+00]]
[[ 1.75645773e-02  1.75645773e-02  1.75645773e-02 -1.57135350e-18
  -2.64372306e-18  5.29455016e-18]]
